### templates 模板
+ 作为Web框架，Django提供了模板，可以很便利的动态生成HTML
+ 模版系统致力于表达外观，而不是程序逻辑
+ 模板的设计实现了业务逻辑(view)与显示内容（template）的分离，一个视图可以使用任意一个模板，一个模板可以供多个视图使用<br><br>

+ 模板包含
+ HTML的静态部分
+ 动态插入内容部分<br><br>

+ Django模板语言，简写DTL，定义在django.template包中
+ 由startproject命令生成的settings.py定义关于模板的值：
+ DIRS定义了一个目录列表，模板引擎按列表顺序搜索这些目录以查找模板源文件
+ APP_DIRS告诉模板引擎是否应该在每个已安装的应用中查找模板
+ 常用方式：在项目的根目录下创建templates目录，设置DIRS值
+ DIRS=[os.path.join(BASE_DIR,"templates")]<br><br>

#### 模板处理
+ Django处理模板分为两个阶段
+ Step1 加载：根据给定的标识找到模板然后预处理，通常会将它编译好放在内存中
+ loader.get_template(template_name)，返回一个Template对象
+ Step2 渲染：使用Context数据对模板插值并返回生成的字符串
+ Template对象的render(RequestContext)方法，使用context渲染模板

In [1]:
# 加载渲染完整代码：
from django.template import loader, RequestContext
from django.http import HttpResponse

def index(request):
    tem = loader.get_template('temtest/index.html')
    context = RequestContext(request, {})
    return HttpResponse(tem.render(context))


ModuleNotFoundError: No module named 'django'

In [ ]:
# 快捷函数
# 为了减少加载模板、渲染模板的重复代码，django提供了快捷函数
render_to_string("")
render(request,'模板',context)
from django.shortcuts import render

def index(request):
    return render(request, 'temtest/index.html')

#### 定义模板
+ 模板语言包括:
+ 变量 
+ 标签 { % 代码块 % }
+ 过滤器
+ 注释{# 代码或html #}
+ 变量
+ 语法：
+ {{ variable }}<br><br>

+ 当模版引擎遇到一个变量，将计算这个变量，然后将结果输出
+ 变量名必须由字母、数字、下划线（不能以下划线开头）和点组成
+ 当模版引擎遇到点(".")，会按照下列顺序查询：
+ 字典查询，例如：foo["bar"]
+ 属性或方法查询，例如：foo.bar
+ 数字索引查询，例如：foo[bar]
+ 如果变量不存在， 模版系统将插入'' (空字符串) 
+ 在模板中调用方法时不能传递参数
+ 使用模板变量时，前面可能是一个字典，可能是一个对象，还可能是一个列表

#### 标签
+ 语法：{ % tag % }
+ 作用
+ 在输出中创建文本
+ 控制循环或逻辑
+ 加载外部信息到模板中供以后的变量使用<br><br>

+ for标签
+ { %for ... in ...%}
+ 循环逻辑
+ {{forloop.counter}}获取当前是第几次循环
+ { %empty%}
+ 给出的列表为或列表不存在时，执行此处
+ { %endfor%}
+ if标签
+ { %if ...%}
+ 逻辑1
+ { %elif ...%}
+ 逻辑2
+ { %else%}
+ 逻辑3
+ { %endif%}<br><br>

+ comment标签
+ { % comment % }
+ 多行注释
+ { % endcomment % }
+ include：加载模板并以标签内的参数渲染
+ { %include "foo/bar.html" % }<br><br>

+ url：反向解析
+ { % url 'name' p1 p2 %}
+ csrf_token：这个标签用于跨站请求伪造保护
+ { % csrf_token %}
+ 布尔标签：and、or，and比or的优先级高
+ block、extends：详见“模板继承”
+ autoescape：详见“HTML转义”

#### 过滤器
+ 用于对模板变量进行操作
+ 语法：{ { 变量|过滤器:参数 }}，例如{ { name|lower }}，表示将变量name的值变为小写输出
+ 使用管道符号 (|)来应用过滤器
+ 通过使用过滤器来改变变量的计算结果
+ 可以在if标签中使用过滤器结合运算符
+ date:改变日期的显示格式
+ lengrh:求长度。字符串,列表
+ default:设置模板变量的默认值

#### 模板注释
+ 单行注释: {# 注释内容 #}
+ 多行注释：{# comment #}
+        注释内容
+        {% endcomment %}

#### 模板继承
+ 模板继承可以减少页面内容的重复定义，实现页面内容的重用
+ 典型应用：网站的头部、尾部是一样的，这些内容可以定义在父模板中，子模板不需要重复定义
+ block标签：在父模板中预留区域，在子模板中填充
+ extends继承：继承，写在模板文件的第一行<br><br>

+ 定义父模板base.html
+ { %block block_name%}
+ 这里可以定义默认值
+ 如果不定义默认值，则表示空字符串
+ { %endblock%}<br><br>

+ 定义子模板index.html
+ { % extends "base.html" %}
+ 在子模板中使用block填充预留区域
+ { %block block_name%}
+ 实际填充内容
+ { %endblock%}<br><br>

##### 说明
+ 如果在模版中使用extends标签，它必须是模版中的第一个标签
+ 不能在一个模版中定义多个相同名字的block标签
+ 子模版不必定义全部父模版中的blocks，如果子模版没有定义block，则使用了父模版中的默认值
+ 如果发现在模板中大量的复制内容，那就应该把内容移动到父模板中
+ 使用可以获取父模板中block的内容
+ 为了更好的可读性，可以给endblock标签一个名字<br><br>

+ { % block block_name %}
+ 区域内容
+ { % endblock block_name %}

#### HTML转义
+ Django对字符串进行自动HTML转义，如在模板中输出如下值：
+ 视图代码：
+ def index(request):
+     return render(request, 'temtest/index2.html',
+                   {
+                       't1': '<h1>hello</h1>'
+                   })
+ 模板代码：
+ {{t1}}
+ 显示效果如下图：
+ 
+ 会被自动转义的字符
+ html转义，就是将包含的html标签输出，而不被解释执行，原因是当显示用户提交字符串时，可能包含一些攻击性的代码，如js脚本
+ Django会将如下字符自动转义：
+ < 会转换为&lt;
+ 
+ > 会转换为&gt;
+ 
+ ' (单引号) 会转换为&#39;
+ 
+ " (双引号)会转换为 &quot;
+ 
+ & 会转换为 &amp;
+ 当显示不被信任的变量时使用escape过滤器，一般省略，因为Django自动转义
+ {{t1|escape}}<br><br>

+ 关闭转义
+ 对于变量使用safe过滤器
+ {{ data|safe }}<br><br>

+ 对于代码块使用autoescape标签
+ { % autoescape off %}
+ {{ body }}
+ { % endautoescape %}
+ 标签autoescape接受on或者off参数<br><br>

+ 自动转义标签在base模板中关闭，在child模板中也是关闭的
+ 字符串字面值
+ 手动转义
+ { { data|default:"<b>123</b>" }}
+ 应写为
+ { { data|default:"&lt;b&gt;123&lt;/b&gt;" }}

### csrf
+ 全称Cross Site Request Forgery，跨站请求伪造
+ 某些恶意网站上包含链接、表单按钮或者JavaScript，它们会利用登录过的用户在浏览器中的认证信息试图在你的网站上完成某些操作，这就是跨站攻击
+ 演示csrf如下
+ 创建视图csrf1用于展示表单，csrf2用于接收post请求
![title](img/csr攻击.png)

#### 防csrf的使用
+ 在django的模板中，提供了防止跨站攻击的方法，使用步骤如下：
+ step1：在settings.py中启用'django.middleware.csrf.CsrfViewMiddleware'中间件，此项在创建项目时，默认被启用
+ step2：在csrf1.html中添加标签

#### 保护原理
+ 查看跨站的信息，并没有cookie信息，即使加入上面的隐藏域代码，发现又可以访问了
+ 结论：django的csrf不是完全的安全得。当提交请求时，中间件'django.middleware.csrf.CsrfViewMiddleware'会对提交的cookie及隐藏域的内容进行验证，如果失败则返回403错误